# Build A (Basic) Random Forest from Scratch

---

> **Note**: this is intended to be a group work lab or a codealong with the instructor.


## What is a Random Forest?

---

Random Forests are some of the most widespread classifiers used. They are relatively simple to use because they require very few parameters to set and they perform well. As we have seen, Decision Trees are very powerful machine learning models.

Decision Trees have some critical limitations. In particular, trees that are grown very deep tend to learn highly irregular patterns: they overfit their training sets. Bagging helps mitigate this problem by exposing different trees to different sub-samples of the whole training set.

Random forests are a further way of averaging multiple deep decision trees, trained on different parts of the same training set, with the goal of reducing the variance. This comes at the expense of a small increase in the bias and some loss of interpretability, but generally greatly boosts the performance of the final model.

### Feature bagging

Random forests differ from bagging decision trees in only one way: they use a modified tree learning algorithm that selects, at each candidate split in the learning process, a random subset of the features. This process is sometimes called feature bagging. 

The reason for doing this is due to correlation of trees in an ordinary bootstrap sample: if one or a few features are very strong predictors for the response variable (target output), these features will be selected in many of the bagging base trees, causing them to become correlated. By selecting a random subset of the features at each split, we avoid this correlation between base trees, strengthening the overall model.

**For a problem with p features, it is typical to use:**
- `p^(1/2)` (rounded down) features in each split for a classification problem.
- `p/3` (rounded down) with a minimum node size of 5 as the default for a regression problem.

## Lab Instructions

---

**A Random Forest classifier is built satisfying these conditions:**
1. Multiple internal decision tree classifiers will be built as the base models.
- For each base model, the data will be resampled with replacement (bootstrapping).
- Each decision tree will be fit on one of the bootstrapped samples of the original data.
- To predict, each internal base model will be passed the new data and make their predictions. The final output will be a vote across the base models for the class.

**Your custom random forest classifier must:**
1. Accept hyperparameters `max_features`, `n_estimators`, and `max_depth`.
2. Implement a `fit` method.
3. Implement a `predict` method.
4. Implement a `score` method.
5. Satisfy the conditions for random forest classifiers listed above!
6. **You will not be implementing feature bagging in this lab, for the sake of simplicity!**

**Test your random forest classifier on the (pre-cleaned) Titanic dataset and compare the performance to a single decision tree classifier!**

> *Note: you are allowed to use the `DecisionTreeClassifier` class in sklearn for the internal base estimators. This lab is about building the random forest ensemble estimator, not building a decision tree class!*

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
titanic = pd.read_csv('../../data/titanic_clean.csv')

FileNotFoundError: File b'../../data/titanic_clean.csv' does not exist

In [3]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [4]:
# This will be the custom, basic version of a Random Forest in the style of
# sklearn's models
class RandomForest(object):
    
    # The __init__ function takes 3 keyword arguments:
    # n_estimators: how many decision tree classifiers are we going to 
    # fit?
    # max_depth: what is the max depth of the internal "base estimator"
    # decision trees?
    # max_features: also a parameter passed to the decision tree classifier
    def __init__(self, n_estimators=3, max_depth=None, max_features=None):
        
        # Set up the 3 keyword arguments as class attributes:
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        
        # create a dictionary that will hold our base estimators
        self.base_estimators = {}
        
        
    # The makes the internal decision tree classifiers
    # Takes data X, y and also an estimator number for
    # assignment to the self.base_estimators dictionary
    def _make_base_estimator(self, X, y, estimator_number):
        
        # make random sample of X, y
        row_indices = list(range(X.shape[0]))
        
        # np.random.choice gives us a random sample of the row
        # indice with replacement
        random_indices = np.random.choice(row_indices, size=len(row_indices),
                                          replace=True)
        
        
        # make "bootstrapped" X, y
        # versions of the data indexed by the randomly sampled row indices
        Xr = X.iloc[random_indices, :]
        yr = y[random_indices]
        
        # Initialize a decision tree classifier with the max_depth
        # and max_features attributes
        dtc = DecisionTreeClassifier(max_depth=self.max_depth,
                                     max_features=self.max_features)
        
        # fit the dtc:
        dtc.fit(Xr, yr)
        
        # put the dtc into our base_estimator dictionary
        self.base_estimators[estimator_number] = dtc
                
    
    # Fitting is actually delegated to the _make_base_estimator function.
    # You specify a number of estimators that you want to fit, and so
    # it iterates through that range, fitting that number of trees
    def fit(self, X, y):
        
        for i in range(self.n_estimators):
            self._make_base_estimator(X, y, i)
            
    
    # Predict calls back into the stored base estimators to get
    # predictions for each. 
    # NOTE: THIS FUNCTION ONLY WORKS FOR THE BINARY 1/0 CLASS
    # PROBLEM!
    def predict(self, X):
        
        # set up votes array with just zeros in it, 
        # the length of rows in X:
        votes = np.zeros(X.shape[0])
        
        # iterate through all the base estimators
        for i in range(self.n_estimators):
            # get out the current one from the dictionary
            base_model = self.base_estimators[i]
            
            # predict with the base estimator on the X matrix
            current_pred = base_model.predict(X)
            
            # add whatever the predictions are to the votes array
            votes = votes + current_pred
            
        # The final output predictions will be 1 any time the vote 
        # count for class 1 was greater than half the number of 
        # base estimators. 
        yhat = (votes >= self.n_estimators/2.).astype(np.int)
        
        return yhat
        
    
    def score(self, X, y):
        
        y = np.array(y)
        
        yhat = self.predict(X)
        
        return ()
        
        

In [5]:
import patsy

In [6]:
y, X = patsy.dmatrices('Survived ~ C(Pclass) + C(Sex) + Age + SibSp + Parch + Fare + C(Embarked) -1',
                      data=titanic, return_type='dataframe')
y = np.ravel(y)

In [7]:
np.mean(y)

0.4044943820224719

In [9]:
rf = RandomForest(n_estimators=1000, max_depth=None, max_features='auto')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.33)

dtc = DecisionTreeClassifier(max_depth=None, max_features='auto')
dtc.fit(Xtrain, ytrain)
print('dtc acc:', dtc.score(Xtest, ytest))

rf.fit(Xtrain, ytrain)
yhat = rf.predict(Xtest)
print('rf acc:', accuracy_score(ytest, yhat))


dtc acc: 0.753191489362
rf acc: 0.8
